In [1]:
from network import Network
from conv import Conv
from dense import Dense
from metrics import log_loss, log_loss_grad, softmax
import math
import numpy as np
import random

In [2]:
random.seed(0)
FILE = "fashion-mnist_train.csv"
with open(f"cnn/images/{FILE}") as f:
    examples = f.read().strip().split("\n")[1:]

train, valid, test = [], [], []
valid_cutoff = .8
test_cutoff = .9

for example in examples:
    label, pixels = example.split(",", 1)
    label = int(label)
    if label > 4:
        continue
    pixels = [int(p) for p in pixels.split(",")]
    data = np.asarray(pixels, dtype="int32")
    data = data.reshape((1,28,28))
    # Scale values from -1 to 1
    data = ((data / 255) - .5) / .5
    # Move channel axis to first axis
    #data = np.moveaxis(data, -1, 0)
    target = np.zeros((1, 5))
    target[0,label] = 1
    row = (data, target, )

    split = random.random()
    if split > test_cutoff:
        test.append(row)
    elif split > valid_cutoff:
        valid.append(row)
    else:
        train.append(row)

In [9]:
class TestNet(Network):
    def __init__(self):
        self.conv1 = Conv(input_channels=1, output_channels=1, kernel_x=3, kernel_y=3)
        self.conv2 = Conv(input_channels=3, output_channels=1, kernel_x=3, kernel_y=3)
        self.dense = Dense(input_size=26 ** 2, output_size=5, relu=False)
        self.last_input = None

        super().__init__()

    def forward(self, x):
        self.last_input = x.copy()
        x = self.conv1.forward(x)
        #x = self.conv2.forward(x)
        x = x.reshape(1, math.prod(x.shape))
        x = self.dense.forward(x)
        return x

    def backward(self, grad, lr):
        prev_hidden = self.conv1.hidden.copy().reshape(1, math.prod(self.conv1.hidden.shape))
        grad = self.dense.backward(grad, lr, prev_hidden)
        #grad = self.conv2.backward(grad, lr)
        grad = self.conv1.backward(grad, lr, self.last_input)

In [11]:
net = TestNet()
lr = 5e-3
epochs = 5

for epoch in range(epochs):
    epoch_loss = 0
    for i, img in enumerate(train):
        image, target = img
        batch = net.forward(image)

        grad = log_loss_grad(softmax(batch), target)
        epoch_loss += log_loss(softmax(batch), target)
        net.backward(grad, lr)

    print(f"Epoch {epoch} loss: {epoch_loss / len(train)}")
    match = np.zeros(len(valid))
    for i, img in enumerate(valid):
        image, target = img
        valid_pred = net.forward(image)
        match[i] = np.argmax(valid_pred) == np.argmax(target)

    print(f"Epoch {epoch} valid accuracy: {match.sum() / match.shape[0]}")

Epoch 0 loss: 0.7330912743388215
Epoch 0 valid accuracy: 0.7805611222444889
Epoch 1 loss: 0.596589137086033
Epoch 1 valid accuracy: 0.8072812291249165
Epoch 2 loss: 0.5335671947706974
Epoch 2 valid accuracy: 0.8183032732130928
Epoch 3 loss: 0.5067247472491165
Epoch 3 valid accuracy: 0.8299933199732799
Epoch 4 loss: 0.4908787078802125
Epoch 4 valid accuracy: 0.8350033400133601
